In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
from tqdm import tqdm
import torch
import sys
import time
import datetime
from itertools import product
import os
import numpy as np
from sklearn import tree
from sklearn import linear_model
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, f1_score
import matplotlib.pyplot as plt
from functools import partial
import shelve
import tqdm
import pickle
from collections import Counter

In [3]:
from ogb.graphproppred import Evaluator
from tdc.benchmark_group import admet_group

/home/michlampert/magisterka/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [17]:
%autoreload 2
from utils import *
from datasets import *
import predict_procedure as pp
import embedding_procedure as ep

In [6]:
# from mat import MolecularAttentionTransformer

# model = MolecularAttentionTransformer()
# model.predict(["CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1"]) ## tooks 11m 4s (!)

In [7]:
# group = admet_group(path = 'data/')
# benchmark = group.get("CYP2D6_Substrate_CarbonMangels")

# dataset = benchmark["test"]

# print(dataset[dataset.Drug == "CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C"].to_markdown())

# Output:
# |     | smiles                             |   Y |
# |----:|:-----------------------------------|----:|
# |  58 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   1 |
# | 226 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   0 |else 'regression'

In [29]:
import pathlib

calculated = set(map(str, pathlib.Path("embeddings/").glob("*/*.zip")))

pairs = product(get_dataset_names(), get_embedder_names())

with open("embedding_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if f"embeddings/{dataset_name}/{model_name}.zip" not in calculated:
            print(dataset_name, model_name, file=f)


In [28]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_classification_names(), ["molbert"]))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs) & set(calculated)), ["forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" not in done:
            print(dataset_name, model_name, head_model, file=f)

In [22]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_dataset_names(), get_embedder_names()))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs) & set(calculated)), ["ridge","forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" in done and dataset_name in get_classification_names():
            print(dataset_name, model_name, head_model, file=f)

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifierCV, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve, auc
from scipy.stats import spearmanr

In [1]:
# %autoreload 2
from molbert.utils.featurizer.molbert_featurizer import MolBertFeaturizer

/home/michlampert/magisterka/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: Enabling RDKit 2023.09.2 jupyter extensions


In [5]:
import torch
import numpy as np

path_to_checkpoint = './models/molbert_100epochs/molbert_100epochs/checkpoints/last.ckpt'
f = MolBertFeaturizer(path_to_checkpoint, device="cpu")

with torch.no_grad():
    features, masks = f.transform('C')

assert all(masks)
print(features)

[[ 0.01830992  0.68017423  0.1749339   0.6642615  -0.9044839   0.01870332
   0.53052753 -0.5770635  -0.8666263   0.7877255  -0.96631885  0.31596518
  -0.8047818  -0.81636214  0.8461241   0.27794883 -0.81167483  0.98382044
   0.89644134 -0.9872448   0.5557583  -0.32108253  0.99881715  0.77729815
  -0.9998408  -0.05704741 -0.7614049  -0.9824723  -0.98456764 -0.571822
  -0.8900104   0.9240311  -0.6789814   0.08562229  0.9700954   0.9999999
   0.9993164   0.7302816   0.05656782 -0.9944156   0.83441573  0.9945194
   0.73471826  0.18875341  0.9840119   0.06622697  0.67733127 -0.8711853
  -0.5281435  -0.55365396  0.9819772   0.9999999  -0.17995857 -0.9996934
   0.99991167 -0.07969973  0.4456461  -0.25901443 -0.9997271   0.78926826
  -0.8225252   0.27110764  0.867051    0.8630969  -0.4926715   0.97550327
   0.7011665   0.29377896  0.99816567  0.8116637   0.2426363  -0.98791575
   0.03659354  0.91715884  0.55181324  0.29384127  0.9935569   0.9819843
  -0.99360514 -0.8353609   0.08793898  0.8131

In [19]:
dataset_name, model_name = "HIA_Hou", "molbert"

df, _ = get_dataset(dataset_name)

model = get_embedder(model_name)

ep.procedure(dataset_name, df, model_name, model, max_samples=10)

Found local copy...
Loading...
Done!
Found local copy...


Processing HIA_Hou with molbert


INFO: Processing HIA_Hou with molbert
100%|██████████| 10/10 [00:04<00:00,  2.34it/s]

Processing HIA_Hou with molbert took 0.1m



INFO: Processing HIA_Hou with molbert took 0.1m


In [20]:
get_embedder("molbert")

<function utils.get_molbert_embedder.<locals>.process(df)>

In [22]:
pd.read_pickle(f"embeddings/{dataset_name}/{model_name}.zip")

FileNotFoundError: [Errno 2] No such file or directory: 'embeddings/HIA_Hou/molbert.zip'

In [46]:
import threeDInfomax.inference

ImportError: cannot import name 'DglPCQM4MDataset' from 'ogb.lsc' (/home/michlampert/magisterka/venv/lib64/python3.10/site-packages/ogb/lsc/__init__.py)

In [36]:

from threeDInfomax.train import load_model

model, _,_ = load_model(args, data=test_data, device=device)
checkpoint = torch.load(args.checkpoint, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model(["C"])

ImportError: cannot import name 'DglPCQM4MDataset' from 'ogb.lsc' (/home/michlampert/magisterka/venv/lib64/python3.10/site-packages/ogb/lsc/__init__.py)